# CSV file 전처리
1. 18세 미만 데이터 삭제
2. Finding Labels가 제1정규화 조건(한 셀에 하나의 값)을 위배
3. 타겟하는 질병라벨만 남기기

In [ ]:
import pandas as pd

# ---------------------------------------------------------
# [Step 0] 예시 데이터 생성 (실제 사용 시 pd.read_csv 사용)
# ---------------------------------------------------------
csv_data="C:\project\lungX\CXR8\Data_Entry_2017_v2020.csv"
df = pd.read_csv(csv_data)



print(f"📌 [원본] 전체 데이터 개수: {len(df)}개")
print(df[['Image Index', 'Patient Age', 'Finding Labels']])
print("-" * 50)

# ---------------------------------------------------------
# [Step 1] 18세 미만 데이터 삭제 (Filtering)
# ---------------------------------------------------------
# 'Patient Age' 컬럼이 숫자형인지 확인 후 필터링
# (만약 '018Y' 같은 문자열 형태라면 숫자 변환 과정이 필요할 수 있습니다)
df_adults = df[df['Patient Age'] >= 18].copy()

print(f"📌 [1단계] 18세 미만 삭제 후 개수: {len(df_adults)}개 (제외된 환자: 10세, 17세)")
print("-" * 50)

# ---------------------------------------------------------
# [Step 2] Finding Labels 제1정규화 (1NF)
# ---------------------------------------------------------
# 1. 파이프('|') 기호를 기준으로 문자열을 리스트로 쪼갭니다.
df_adults['Finding Labels'] = df_adults['Finding Labels'].str.split('|')

# 2. explode() 함수로 리스트 요소를 행으로 펼칩니다.
df_final = df_adults.explode('Finding Labels')

# 3. 인덱스 정리
df_final = df_final.reset_index(drop=True)

print(f"📌 [2단계] 제1정규화(Explode) 완료 개수: {len(df_final)}개")
print("   (다중 라벨이 개별 행으로 분리됨)")
print("-" * 50)

# 결과 확인
print(df_final[['Image Index', 'Patient Age', 'Finding Labels']])



# ---------------------------------------------------------
# [Step 3] 특정 6개 라벨만 남기기 (Filtering)
# ---------------------------------------------------------

# 1. 남길 라벨 목록 정의 (Allow List)
target_labels = [
    'No Finding', 
    'Infiltration', 
    'Effusion', 
    'Atelectasis', 
    'Nodule', 
    'Mass'
]

# 2. 필터링 수행
# df_final은 이전 단계(제1정규화)에서 생성된 데이터프레임이라고 가정합니다.
# 'Finding Labels' 컬럼의 값이 target_labels 안에 있는 경우만 남깁니다.
df_filtered = df_final[df_final['Finding Labels'].isin(target_labels)].copy()

# 3. 인덱스 재정렬
df_filtered = df_filtered.reset_index(drop=True)

# ---------------------------------------------------------
# [결과 확인]
# ---------------------------------------------------------
print(f"📌 [필터링 전] 데이터 개수: {len(df_final)}개")
print(f"📌 [필터링 후] 데이터 개수: {len(df_filtered)}개")
print("-" * 50)

# 어떤 라벨들이 남았는지 확인 (중복 제거 후 출력)
unique_labels = df_filtered['Finding Labels'].unique()
print(f"✅ 현재 남아있는 라벨 목록 ({len(unique_labels)}종):")
print(unique_labels)

# 데이터 미리보기
print("-" * 50)
print(df_filtered.head())



# 파일 저장
df_filtered.to_csv("Data_Entry_processed.csv", index=False)


### 타겟질병 종류 변화에 따른 데이터 라벨 삭제

- 타겟 질병 5개 -> 3개 로 변화함.
- 해당 코드는 두개의 질병은 Nodule과 Mass를 삭제.

In [ ]:
import pandas as pd

# ==========================================
# 1. 파일 경로 설정
# ==========================================
# 원본 CSV 경로
INPUT_CSV_PATH = r"C:\project\lungX\CXR8\Data_Entry_processed.csv"
# 수정된 CSV를 저장할 경로 (안전을 위해 다른 이름으로 저장 권장)
OUTPUT_CSV_PATH = r"C:/project/lungX/CXR8/Data_Entry_processed_Final.csv"

# 삭제할 라벨 목록
DELETE_LABELS = ['Mass', 'Nodule']

# ==========================================
# 2. 데이터 로드 및 삭제 처리
# ==========================================
print("CSV 파일을 읽는 중...")
try:
    df = pd.read_csv(INPUT_CSV_PATH)
    
    # 컬럼명 및 데이터 공백 제거 (안전장치)
    df.columns = df.columns.str.strip()
    df['Finding Labels'] = df['Finding Labels'].str.strip()
    
    original_count = len(df)
    print(f"원본 데이터 개수: {original_count}개")

    # 🚨 핵심 로직: 삭제할 라벨에 포함되지 않는(~Not) 행만 남김
    # isin(DELETE_LABELS) -> Mass나 Nodule인 행을 찾음
    # ~ (Tilde) -> 그 반대인 행들만 선택 (즉, 삭제 대상 제외하고 남김)
    df_filtered = df[~df['Finding Labels'].isin(DELETE_LABELS)]
    
    filtered_count = len(df_filtered)
    deleted_count = original_count - filtered_count

    print("-" * 30)
    print(f"삭제된 라벨: {DELETE_LABELS}")
    print(f"삭제된 행 개수: {deleted_count}개")
    print(f"남은 데이터 개수: {filtered_count}개")
    print("-" * 30)

    # ==========================================
    # 3. 결과 저장
    # ==========================================
    # index=False는 0, 1, 2... 같은 불필요한 인덱스 번호를 저장하지 않음
    df_filtered.to_csv(OUTPUT_CSV_PATH, index=False)
    print(f"✅ 저장 완료! 경로: {OUTPUT_CSV_PATH}")

except FileNotFoundError:
    print("❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")

# IMG file 전처리

## Finding Label에 없는 이미지 데이터 삭제
- 사실 이건 데이터 셀렉 먼저 했으면 할 필요 없는 작업이엇죠
바보엿죠 제가



In [1]:
import os
import pandas as pd

In [ ]:
def delete_unlisted_images(csv_path, image_folder_path):
    # ---------------------------------------------------------
    # [Step 1] CSV에서 유효한 파일 목록 로드
    # ---------------------------------------------------------
    # CSV 파일 읽기
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print("❌ CSV 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        return

    # 'Image Index' 컬럼을 set(집합)으로 변환
    # (set은 list보다 검색 속도가 훨씬 빨라 수만 장의 파일 비교에 유리합니다)
    if 'Image Index' not in df.columns:
        print("❌ CSV 내에 'Image Index' 컬럼이 없습니다.")
        return
        
    valid_files_set = set(df['Image Index'].values)
    
    print(f"📋 CSV 기준 유효한 이미지 수: {len(valid_files_set)}개")

    # ---------------------------------------------------------
    # [Step 2] 폴더 스캔 및 삭제 수행
    # ---------------------------------------------------------
    if not os.path.exists(image_folder_path):
        print("❌ 이미지 폴더가 존재하지 않습니다.")
        return

    # 폴더 내 모든 파일 리스트 가져오기
    all_files = os.listdir(image_folder_path)
    
    deleted_count = 0
    kept_count = 0
    
    print("🚀 불필요한 파일 삭제 시작...")

    for filename in all_files:
        # PNG 파일만 대상으로 함 (다른 시스템 파일 삭제 방지)
        if filename.lower().endswith('.png'):
            
            # [삭제 조건] 폴더에 있는 파일명이 CSV 리스트(valid_files_set)에 없다면?
            if filename not in valid_files_set:
                file_path = os.path.join(image_folder_path, filename)
                
                try:
                    os.remove(file_path) # <--- 실제 삭제 명령
                    # print(f"🗑️ 삭제됨: {filename}") # 너무 많이 출력되면 주석 처리
                    deleted_count += 1
                except Exception as e:
                    print(f"⚠️ 삭제 실패 ({filename}): {e}")
            else:
                kept_count += 1

    # ---------------------------------------------------------
    # [Step 3] 결과 리포트
    # ---------------------------------------------------------
    print("=" * 50)
    print("🎉 정리 완료")
    print(f"✅ 유지된 파일 (CSV에 있음): {kept_count}개")
    print(f"🗑️ 삭제된 파일 (CSV에 없음): {deleted_count}개")
    print("=" * 50)






In [4]:

# --- 실행 설정 ---
# 1. 필터링된 CSV 파일 경로 (앞서 만든 filtered_data_6_classes.csv 등)
target_csv_path = "C:\project\lungX\CXR8\Data_Entry_processed.csv" 

# 2. 이미지가 들어있는 실제 폴더 경로
target_image_folder = "D:\lung_xray\processed2_imgs"

# 함수 실행
delete_unlisted_images(target_csv_path, target_image_folder)

📋 CSV 기준 유효한 이미지 수: 97675개
🚀 불필요한 파일 삭제 시작...
🎉 정리 완료
✅ 유지된 파일 (CSV에 있음): 97675개
🗑️ 삭제된 파일 (CSV에 없음): 14445개


### PA 이미지만 따로 폴더 생성
- 목적 : 빠르게 실행해서 확인해볼 샘플

In [2]:
import pandas as pd
import os
import shutil

In [3]:
# ==========================================
# 1. 경로 설정
# ==========================================
CSV_PATH = r"C:\project\lungX\CXR8\Data_Entry_processed.csv"
SOURCE_DIR = r"D:\lung_xray\processed2_imgs"  # 원본 이미지가 모여있는 폴더
TARGET_DIR = r"D:\lung_xray\final_imgsets"  # 결과물을 저장할 새 폴더

# ==========================================
# 2. PA 데이터 리스트 확보 (CSV 읽기)
# ==========================================
print("데이터 로드 및 분류 목록 생성 중...")
try:
    df = pd.read_csv(CSV_PATH)
    df.columns = df.columns.str.strip()
    
    # View Position이 'PA'인 파일명들만 추출하여 집합(Set)으로 만듦
    # (Set을 사용해야 검색 속도가 매우 빠릅니다)
    pa_file_set = set(df[df['View Position'] == 'PA']['Image Index'].values)
    
    print(f"CSV 기준 전체 PA 라벨 데이터 수: {len(pa_file_set)}개")

except Exception as e:
    print(f"❌ CSV 로드 실패: {e}")
    exit()

# ==========================================
# 3. 파일 복사 실행
# ==========================================
# 타겟 폴더가 없으면 생성
os.makedirs(TARGET_DIR, exist_ok=True)

if not os.path.exists(SOURCE_DIR):
    print("❌ 원본 이미지 폴더를 찾을 수 없습니다.")
    exit()

all_files = os.listdir(SOURCE_DIR)
total_files = len(all_files)

print(f"\n원본 폴더 파일 수: {total_files}개")
print(f"복사 작업 시작... ('{SOURCE_DIR}' -> '{TARGET_DIR}')")

copied_count = 0
skipped_count = 0  # PA가 아닌 파일들
processed_count = 0

for filename in all_files:
    src_path = os.path.join(SOURCE_DIR, filename)
    
    # 1. 파일이 아니면(폴더 등) 건너뜀
    if not os.path.isfile(src_path):
        continue
        
    processed_count += 1
    
    # 2. 현재 파일이 PA 목록(Set)에 있는지 확인
    if filename in pa_file_set:
        dst_path = os.path.join(TARGET_DIR, filename)
        
        try:
            # 원본 유지하며 복사
            shutil.copy(src_path, dst_path)
            copied_count += 1
        except Exception as e:
            print(f"❌ 복사 에러 ({filename}): {e}")
            
    else:
        # PA가 아닌 경우 (AP 등) -> 그냥 건너뜀 (삭제 X)
        skipped_count += 1

    # 3. 500장마다 진행상황 출력
    if processed_count % 500 == 0:
        print(f"[{processed_count}/{total_files}] 처리 중... (복사됨: {copied_count}, 건너뜀: {skipped_count})")

# ==========================================
# 4. 완료 리포트
# ==========================================
print("\n=== 작업 완료 ===")
print(f"저장된 폴더: {TARGET_DIR}")
print(f"총 복사된 PA 이미지: {copied_count}장")
print(f"제외된 이미지 (AP 등): {skipped_count}장")

데이터 로드 및 분류 목록 생성 중...
CSV 기준 전체 PA 라벨 데이터 수: 59176개

원본 폴더 파일 수: 97676개
복사 작업 시작... ('D:\lung_xray\processed2_imgs' -> 'D:\lung_xray\final_imgsets')
[500/97676] 처리 중... (복사됨: 317, 건너뜀: 183)
[1000/97676] 처리 중... (복사됨: 638, 건너뜀: 362)
[1500/97676] 처리 중... (복사됨: 1009, 건너뜀: 491)
[2000/97676] 처리 중... (복사됨: 1306, 건너뜀: 694)
[2500/97676] 처리 중... (복사됨: 1684, 건너뜀: 816)
[3000/97676] 처리 중... (복사됨: 1975, 건너뜀: 1025)
[3500/97676] 처리 중... (복사됨: 2319, 건너뜀: 1181)
[4000/97676] 처리 중... (복사됨: 2668, 건너뜀: 1332)
[4500/97676] 처리 중... (복사됨: 2939, 건너뜀: 1561)
[5000/97676] 처리 중... (복사됨: 3252, 건너뜀: 1748)
[5500/97676] 처리 중... (복사됨: 3585, 건너뜀: 1915)
[6000/97676] 처리 중... (복사됨: 3837, 건너뜀: 2163)
[6500/97676] 처리 중... (복사됨: 4163, 건너뜀: 2337)
[7000/97676] 처리 중... (복사됨: 4551, 건너뜀: 2449)
[7500/97676] 처리 중... (복사됨: 4912, 건너뜀: 2588)
[8000/97676] 처리 중... (복사됨: 5207, 건너뜀: 2793)
[8500/97676] 처리 중... (복사됨: 5554, 건너뜀: 2946)
[9000/97676] 처리 중... (복사됨: 5896, 건너뜀: 3104)
[9500/97676] 처리 중... (복사됨: 6259, 건너뜀: 3241)
[10000/97676] 처리 중... 

### Finding Labels가 Mass와 Nodule인 IMG 삭제


In [4]:
import pandas as pd
import os

# ==========================================
# 1. 경로 설정
# ==========================================
CSV_PATH = r"C:\project\lungX\CXR8\Data_Entry_processed.csv"
TARGET_DIR = r"D:\lung_xray\final_imgsets"

# 삭제할 라벨 정확히 지정
DELETE_LABELS = ['Nodule', 'Mass']

# ==========================================
# 2. 삭제 대상 리스트 확보
# ==========================================
print("데이터 로드 중...")
try:
    df = pd.read_csv(CSV_PATH)
    df.columns = df.columns.str.strip()
    
    # 🚨 전제 조건: 라벨이 하나만 존재함 (Multi-label 없음)
    # Finding Labels 컬럼의 값이 'Nodule' 이거나 'Mass' 인 행만 추출
    target_df = df[df['Finding Labels'].isin(DELETE_LABELS)]
    
    # 삭제할 파일명들을 집합(Set)으로 저장
    delete_targets = set(target_df['Image Index'].values)
    
    print(f"삭제 대상 라벨(Nodule, Mass) 데이터 수: {len(delete_targets)}개")

except Exception as e:
    print(f"❌ CSV 로드 실패: {e}")
    exit()

# ==========================================
# 3. 삭제 실행 (로그 기능 포함)
# ==========================================
if not os.path.exists(TARGET_DIR):
    print(f"❌ 폴더가 없습니다: {TARGET_DIR}")
    exit()

all_files = os.listdir(TARGET_DIR)
total_files = len(all_files)

print(f"\n폴더({TARGET_DIR}) 정리 시작...")
print(f"총 파일 수: {total_files}개")

deleted_count = 0
kept_count = 0

# enumerate를 사용하여 인덱스(i)와 파일명(filename)을 같이 꺼냅니다.
for i, filename in enumerate(all_files):
    file_path = os.path.join(TARGET_DIR, filename)
    
    # 파일이 아니면 건너뜀
    if not os.path.isfile(file_path):
        continue

    try:
        # 삭제 리스트에 있으면 삭제
        if filename in delete_targets:
            os.remove(file_path)
            deleted_count += 1
        else:
            kept_count += 1
            
        # ----------------------------------------------------
        # 📌 요청하신 500장 단위 로그 출력 기능
        # ----------------------------------------------------
        if (i + 1) % 500 == 0:
            print(f"[System Log] 현재 {i + 1} / {total_files} 장 처리 완료... (삭제: {deleted_count})")
            
    except Exception as e:
        print(f"[Error] {filename}: {e}")

# ==========================================
# 4. 결과 리포트
# ==========================================
print("\n=== 작업 완료 ===")
print(f"총 처리된 파일: {i + 1}장")
print(f"삭제된 파일 (Nodule/Mass): {deleted_count}장")
print(f"남은 파일: {kept_count}장")

데이터 로드 중...
삭제 대상 라벨(Nodule, Mass) 데이터 수: 10857개

폴더(D:\lung_xray\final_imgsets) 정리 시작...
총 파일 수: 59176개
[System Log] 현재 500 / 59176 장 처리 완료... (삭제: 59)
[System Log] 현재 1000 / 59176 장 처리 완료... (삭제: 100)
[System Log] 현재 1500 / 59176 장 처리 완료... (삭제: 135)
[System Log] 현재 2000 / 59176 장 처리 완료... (삭제: 178)
[System Log] 현재 2500 / 59176 장 처리 완료... (삭제: 223)
[System Log] 현재 3000 / 59176 장 처리 완료... (삭제: 258)
[System Log] 현재 3500 / 59176 장 처리 완료... (삭제: 302)
[System Log] 현재 4000 / 59176 장 처리 완료... (삭제: 332)
[System Log] 현재 4500 / 59176 장 처리 완료... (삭제: 368)
[System Log] 현재 5000 / 59176 장 처리 완료... (삭제: 408)
[System Log] 현재 5500 / 59176 장 처리 완료... (삭제: 447)
[System Log] 현재 6000 / 59176 장 처리 완료... (삭제: 480)
[System Log] 현재 6500 / 59176 장 처리 완료... (삭제: 503)
[System Log] 현재 7000 / 59176 장 처리 완료... (삭제: 534)
[System Log] 현재 7500 / 59176 장 처리 완료... (삭제: 590)
[System Log] 현재 8000 / 59176 장 처리 완료... (삭제: 629)
[System Log] 현재 8500 / 59176 장 처리 완료... (삭제: 663)
[System Log] 현재 9000 / 59176 장 처리 완료... (삭제: 71